2018 / 2 / 01

Wayne Nixalo

# RetinaNet Finetuning

## 1. Imports & Initialization

In [ ]:
import keras
import keras.preprocessing.image

import keras_retinanet.losses
import keras_retinanet.layers
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.callbacks import RedirectModel
from keras_retinanet.models.resnet import ResNet50RetinaNet
from keras_retinanet.utils.keras_version import check_keras_version
from keras_retinanet.utils.image import preprocess_image, resize_image
from keras_retinanet.models.resnet import custom_objects # <-- what do I need this for?

import tensorflow as tf

import numpy as np
import os
import cv2

In [ ]:
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

Taken from [Boring Detector train.py script](https://github.com/lexfridman/boring-detector/blob/master/train.py#L43)

Based on [initialization API from RetinaNet](https://github.com/fizyr/keras-retinanet/blob/master/keras_retinanet/models/resnet.py#L100)

In [ ]:
def create_models(num_classes):
    # create "base" model (no NMS)
    image = keras.layers.Input((None, None, 3))
    
    model = ResNet50RetinaNet(image, num_classes=num_classes, weights='imagenet', nms=False)
    training_model = model
    
    # append NMS for prediction only
    classification   = model.outputs[1]
    detections       = model.outputs[2]
    boxes            = keras.layers.Lambda(lambda x: x[:, :, :4])(detections)
    detections       = keras_retinanet.layers.NonMaximumSuppression(name='nms')([boxes, classification, detections])
    prediction_model = keras.models.Model(inputs=model.inputs, outputs=model.outputs[:2] + [detections])
    
    # compile model
    training_model.compile(
        loss={
            'regression'    : keras_retinanet.losses.smooth_l1(),
            'classification':keras_retinanet.losses.focal()
        },
        optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
    )
    
    return model, training_model, prediction_model

In [ ]:
# keras.backend.tensorflow_backend.set_session(get_session())

# model = keras.models.load_model('data/retinanet-model/resnet50_coco_best_v1.2.2.h5',
#                                     custom_objects=custom_objects)

In [ ]:
LENDATSET = 7637 # length of dataset (7637 images)
batch_size = 1

num_classes = 1
steps_per_epoch = LENDATSET // batch_size # may need edits for batch sizes that leave remainders

In [ ]:
image_gen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True)
datagen = CSVGenerator(csv_data_file='data/interstage_retinanet_data.csv',
                       csv_class_file='data/interstage_retinanet_classes.csv',
                       image_data_generator=image_gen,
                       batch_size=batch_size)

In [ ]:
model, trainer_model, predictor_model = create_models(num_classes)
# dont yet exactly know why it's done this way: 3 returned 'model' versions

## 2. Training

In [ ]:
trainer_model.fit_generator(generator = datagen,
                            steps_per_epoch = steps_per_epoch, # eh?
                            epochs    = 1,
                            verbose   = 1,)

## 3. Testing & Visualization

In [ ]:
test_image_path = 'data/interstage_train/000000-000412/000001.jpg' # or script for random impath
test_image = cv2.imread(test_image_path)

prediction = predictor_model.predict_on_batch(np.expand_dims(test_image, axis=0))

In [ ]:
# preprocess image for neural network
image = preprocess_image(test_image)
image, scale = resize_image(image)

# detect on image
_,_,detections = model.predict_on_batch(np.expand_dims(image, axis=0))

# compute predicted labels and scores
detected_label = np.argmax(detections[0,:,4:], axis=1)
scores         = detections[0, np.arange(detections.shape[1]), 4 + detected_label]

# correct for image scale
detections[0,:,:4] /= scale

In [ ]:
bbx = detections[0, 0, :4]
bbx